In [21]:
!pip install biom-format


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [22]:
from biom import load_table
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report
from sklearn.tree import export_text
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import chi2,SelectKBest
from sklearn.linear_model import ElasticNet
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [23]:
import os
path = os.getcwd()

In [24]:
import pandas as pd

# Assuming your data is in a file named "data.xlsx" in the same directory
data = pd.read_csv(path + '/Data/tax_defined_observations.tsv', skiprows=2)
#tax_defined_observations.tsv


In [25]:
# Load the BIOM file
biom_file = path + '/Data/tax_defined_observations.tsv'
table = load_table(biom_file)

In [26]:
table

986 x 951 <class 'biom.table.Table'> with 76533 nonzero entries (8% dense)

In [32]:
data = pd.DataFrame(table.to_dataframe())

In [36]:
data['taxa'] = data.index
print(data['taxa'])

k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Lachnospiraceae; g__Blautia; s__                       k__Bacteria; p__Firmicutes; c__Clostridia; o__...
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Lachnospiraceae; g__; s__                              k__Bacteria; p__Firmicutes; c__Clostridia; o__...
k__Bacteria; p__Firmicutes; c__Bacilli; o__Turicibacterales; f__Turicibacteraceae; g__Turicibacter; s__                k__Bacteria; p__Firmicutes; c__Bacilli; o__Tur...
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Ruminococcaceae; g__; s__                              k__Bacteria; p__Firmicutes; c__Clostridia; o__...
k__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Lactobacillaceae; g__Lactobacillus; s__coleohominis     k__Bacteria; p__Firmicutes; c__Bacilli; o__Lac...
                                                                                                                                             ...           

In [39]:
# Split the "taxa" column on the semicolon and expand it into multiple columns
data[['kingdom', 'phylum', 'class', 'order', 'family', 'genum', 'species']] = data['taxa'].str.split(';', expand=True)

# Remove the leading "__" and trailing whitespace from each cell in the new columns
data['kingdom'] = data['kingdom'].str.strip().str.replace('k__', '')
data['phylum'] = data['phylum'].str.strip().str.replace('p__', '')
data['class'] = data['class'].str.strip().str.replace('c__', '')
data['order'] = data['order'].str.strip().str.replace('o__', '')
data['family'] = data['family'].str.strip().str.replace('f__', '')
data['genum'] = data['genum'].str.strip().str.replace('g__', '')
data['species'] = data['species'].str.strip().str.replace('s__', '')

# Print the updated dataframe with new columns
# print(data)
data.to_excel('Data/tax_defined_observations_updated.xlsx', index=False)

print("New Excel file 'updated_data.xlsx' created with the updated data.")

New Excel file 'updated_data.xlsx' created with the updated data.


In [40]:
print(data)


                                                    10317.000043142.57559  \
k__Bacteria; p__Firmicutes; c__Clostridia; o__C...                  755.0   
k__Bacteria; p__Firmicutes; c__Clostridia; o__C...                  749.0   
k__Bacteria; p__Firmicutes; c__Bacilli; o__Turi...                    0.0   
k__Bacteria; p__Firmicutes; c__Clostridia; o__C...                 2572.0   
k__Bacteria; p__Firmicutes; c__Bacilli; o__Lact...                    0.0   
...                                                                   ...   
k__Bacteria; p__Firmicutes; c__Bacilli; o__Lact...                    0.0   
k__Bacteria; p__Chloroflexi; c__C0119; o__; f__...                    0.0   
k__Bacteria; p__Proteobacteria; c__Gammaproteob...                    0.0   
k__Bacteria; p__Firmicutes; c__Bacilli; o__Lact...                    0.0   
k__Bacteria; p__Proteobacteria; c__Betaproteoba...                    0.0   

                                                    10317.000038011.57180  

In [41]:
#load metadata
metadata = path + '/Data/metadata_filtered.txt'
md = pd.read_csv(metadata, sep = '\t')
md

,#SampleID,subset_diabetes,host_taxid,allergic_to_unspecified,qiita_study_id,host_common_name,non_food_allergies_sun,physical_specimen_remaining,alcohol_types_unspecified,body_site,...,allergic_to_i_have_no_food_allergies_that_i_know_of,taxon_id,subset_bmi,description,non_food_allergies_drug_eg_penicillin,subset_healthy,env_feature,body_habitat,public,sex
0,10317.000074244.69032,True,9606.0,No,10317.0,human,No,Yes,No,UBERON:feces,...,Yes,408170.0,False,American Gut Project Stool sample,No,False,human-associated habitat,UBERON:feces,Yes,female
1,10317.000097250.74745,True,9606.0,No,10317.0,human,No,Yes,No,UBERON:feces,...,Yes,408170.0,True,American Gut Project Stool sample,No,False,human-associated habitat,UBERON:feces,Yes,male
2,10317.000037974.57180,True,9606.0,TRUE,10317.0,human,FALSE,TRUE,FALSE,UBERON:feces,...,FALSE,408170.0,True,American Gut Project Stool Sample,TRUE,True,human-associated habitat,UBERON:feces,TRUE,male
3,10317.000110314.76911,False,9606.0,No,10317.0,human,No,Yes,Yes,UBERON:feces,...,Yes,408170.0,False,American Gut Project Stool sample,No,False,human-associated habitat,UBERON:feces,Yes,female
4,10317.000051141.58828,True,9606.0,FALSE,10317.0,human,FALSE,TRUE,FALSE,UBERON:feces,...,FALSE,408170.0,True,American Gut Project Stool Sample,FALSE,False,human-associated habitat,UBERON:feces,TRUE,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,10317.000054290.59891,True,9606.0,FALSE,10317.0,human,FALSE,TRUE,FALSE,UBERON:feces,...,TRUE,408170.0,True,American Gut Project Stool Sample,FALSE,True,human-associated habitat,UBERON:feces,TRUE,female
1215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
md["cardiovascular_disease"] = md["cardiovascular_disease"].map({"Diagnosed by a medical professional (doctor, physician assistant)": 1, "I do not have this condition": 0})
print(md["cardiovascular_disease"])

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
1214    0.0
1215    NaN
1216    NaN
1217    NaN
1218    NaN
Name: cardiovascular_disease, Length: 1219, dtype: float64


In [43]:
# Convert the table to a DataFrame
# data = pd.DataFrame(table.matrix_data.toarray(),
#                     index=table.ids(axis='observation'),
#                     columns=table.ids(axis='sample'))



data = pd.DataFrame(table.matrix_data.toarray().T, columns=table.ids(axis='observation'))
data['SampleID'] = table.ids(axis='sample')
data = data.sort_values(by='SampleID')
md = md.sort_values(by='#SampleID')
md = md.reset_index(drop=True)

# Drop duplicate SampleID values from data
data = data.drop_duplicates('SampleID')
md = md.drop_duplicates('#SampleID')


data['CVD'] = data['SampleID'].map(md.set_index('#SampleID')['cardiovascular_disease'])

data
# Display the updated DataFrame
final_data = data.drop('SampleID',axis =1)

In [44]:
# Split the data into features (X) and target variable (y)
X = final_data.drop('CVD', axis=1)
y = final_data['CVD']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [45]:
#Create a decision tree classifier
clf = DecisionTreeClassifier()

# Fit the classifier to the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate and print the classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

# Extract specificity and sensitivity from the classification report
specificity = report.split()[-4]
sensitivity = report.split()[-2]

# Print specificity and sensitivity
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)

Accuracy: 0.5979020979020979
Classification Report:
              precision    recall  f1-score   support

         0.0       0.58      0.56      0.57       135
         1.0       0.62      0.63      0.62       151

    accuracy                           0.60       286
   macro avg       0.60      0.60      0.60       286
weighted avg       0.60      0.60      0.60       286

Specificity: 0.60
Sensitivity: 0.60
